# Create an error-correcting DDM workflow

This notebook documents the workflow used to identify, analyze and correct the errors associated with using a physically based model (MODFLOW) for hydrological (groundwater/surface-water) simulation.

Based on DDM-UQ (Data Driven Model-Uncertainty Quantification) framework specified in Xu and Valocchi, 2015.

This notebook uses the Python version of the MATLAB based DDM-UQ toolbox.


## References

### DDM:

* https://wiki.illinois.edu/wiki/display/mlgwm/Home

* T. Xu, A. J. Valocchi, 2015. Data-driven Methods to Improve Baseflow Prediction of a Regional Groundwater Model. Computers and Geociences. doi: 10.1016/j.cageo.2015.05.016

* T. Xu, A. J. Valocchi, J. Choi, E. Amir, 2013. Use of Machine Learning Methods to Reduce Predictive Error of Groundwater Models. Groundwater. doi: 10.1111/gwat.12061


## Contact author:

Rishi Jumani; unbiased.modeler@gmail.com

# Main concepts


## Account for three main types of errors

* Structural errors

* Paramter errors

* Measurement errors

## Advantages of using DDM:

* Works for errors from multiple sources, and does not make any assumptions about the error distribution

* May be possible for the DDM to generalize to new hydrogeological conditions, that differ from the training data



## Disadvantages of using DDM:

* Requires existence of structural errors (spatial and temporal patterns). Might not be suitable for well calibrated models (calibration error follows Gaussian distribution with zero mean and variance similar to observation error). - ** This workflow attempts to address this disadvantage by including more input features than considered in the papers  **

* DDM may not conserve mass


## Residual Analysis

* Learn a relation between the residuals (errors) and the input factors.

* Model the spatio-temporal residuals - uncover the bias, correlation, heteroscedasticity in the results, by capturing the systematic patterns in the residuals.



## Machine Learning

### Support Vector Machine (SVM)

* Goal: Find a relation between the residual $\epsilon$ and selected model inputs $\mathbf{x}$:

### $$\epsilon = \hat{\epsilon}(\mathbf{x})$$

* Train the DDM with the help of a test dataset.

* Has good generalization performance, and finds the global minima.

* Minimize the upper bound of the generalization error rather than minimizing the training error



## Error Variance Analysis (EVA)




## Structural Errors






## Further work

* Build local DDM (based on clustering the residuals, so that residuals in the same cluster have similar characteristics)








## Freyberg Model

## Model Calibration

### Calibrated Parameters:

* Hydraulic Conductivity


### Observations used for Calibration:

* Head measurements

* Baseflow

### Calibration Technique

* Pilot Points


### Software used

pyEMU and PEST++

## Residual Analysis

* Compute Mean Error (ME) of the head (negative value if simulated head $>$ observed head): if magnitude of ME is large, this would indicate bias in the residuals

* Construct a semivariogram to examine the spatial correlation in the residual

* Use residual plots to estimate bias and heteroscedasticity

* Use Q-Q plot to test for Gaussianity

* Use the Durbin-Watson (DW) test to examine the  temporal correlation (auto-correlation) in the residual

### Analysis of semivariogram

* If the variance of the difference between the residuals at two wells increases as the lag distance increases within the range, this would indicate the existence of spatial dependence

* What is the nugget effect and what is the range? (More prominent nugget effect and smaller range indicate that spatial correlation of error is not as strong)

* Is the density of monitoring wells sufficient: if spatial correlation grows weaker as distance increases, then it is not

### Residual Plots

### Q-Q plot

### Analysis of DW statistic

The DW statistic tests for $1^{st}$ order auto-correlation

The DW statistic works with time series data with uniform time intervals: only include wells that have relatively uniformly spaced water level measurements

If the DW statistic if a majority of the wells is $<$ 2, this would indicate that the residuals are correlated in time

### Compute Average Mututal Information scores

To check if the historical residuals are related to the well location, simulated head or time of measuremenet

In [2]:
a = 2

In [3]:
#  %%javascript
#     MathJax.Hub.Config({
#        TeX: { equationNumbers: { autoNumber: "AMS" } }
#      });


## Train the SVM

### Software used: scikit-learn

### Ways to apply the DDM:

* Temporal Prediction during prediction period

* Spatial Predicition

* Spatial-Temporal Prediction


### Basic Theory

* Project the input $\mathbf{x}$ to a higher dimensional feature space with the map $\phi : \mathcal{X} \rightarrow \mathcal{F}$

* Carry out a linear regression in the feature space $\phi(\mathbf{x})$:

$$ f(\mathbf{x}) = \mathbf{w} \cdot \phi (\mathbf{x}) + b $$

* estimate the coefficients $\mathbf{w}$ and b by solving an optimization problem:

$$ \begin{equation} \label{one} 
\mbox{ minimize } \frac{1}{2} || \mathbf{w} ||^2 + C \sum_{i=1}^{n} (\xi_i + \xi_i^*)
\end{equation} $$

$$ \begin{equation} \label{two} 
\mbox{ subject to } (\mathbf{w}^T \phi(\mathbf{x}_i) + b) - y_i \le \varepsilon + \xi_i
\end{equation} $$

$$\begin{equation} \label{three} 
y_i - \left( \mathbf{w}^T \phi(\mathbf{x}_i) + b  \right) \le \varepsilon + \xi_i^*
\end{equation} $$

$$ \begin{equation} \label{four} 
\xi_i, \xi_i^* \ge 0, \mbox{  } i = 1,.., n
\end{equation} $$

* The first term in \eqref{one} represents the complexity of the regression model, and acts as regularization.

* The second term in \eqref{one} represents the goodness-of-fit of the training data

* The slack variables $\xi_i, a\xi_i^*$ are used to cope with infeasible constraints of the optimization problem, and are derived from the $\varepsilon$-insensitive loss function $|\xi|_\varepsilon = max \{0, |y_i - f(\mathbf{x}_i| - \varepsilon \} $

* The constant C, known as the regularization hyper-parameter, in \eqref{one} determines the trade-off between the flatness of f and the deviations exceeding $\varepsilon$

* The map $\phi : \mathcal{X} \rightarrow \mathcal{F}$ is implemented via kernel functions:

$$ \langle \phi (\mathbf{x}_i) , \phi (\mathbf{x}_j) \rangle = K(\mathbf{x}_i, \mathbf{x}_j) $$

* Use an RBF (Radial Basis Function) kernel:

$$ K(\mathbf{x}_i, \mathbf{x}_j) = exp(-\gamma \mbox{  } || \mathbf{x}_i - \mathbf{x}_j ||^2)   $$

### Choose input data

* For temporal and spatio-temporal prediction scenarios, that require the extrapolation of time, including time as an input feature leads to inferior performance of the DDM, while for **spatial prediction**, including time as an input feature yields better performance.

#### Input features for temporal and spatio-temporal predictions:

* Well location $(x_w, y_w)$ of observations

* Simulated head, $\hat{h}$


#### Input features for spatial predictions:

* In addition to the above two factors, also includes time


#### Input parameters to consider:

* Pumping rates

* Boundary Conditions

* Month

* location of observations

* Precip rates

* ET

* difference between groundwater head and streambed elevation

* simulated baseflow


### Selection criteria

* Add input parameters one by one, and accept a parameter if CV error decreases after adding it

### Divide the data into Training and Testing subsets

### Tune the parameters

* Use k-fold cross validation (CV); k chosen as 5, 10

* Data was partitioned differently for spatial and spatio-temporal scenarios, and for temporal scenario

### Parameters to be tuned:

* regularization hyper-parameter, C (determines the complexity of the DDM) 

* loss function parameter $\varepsilon$

* Kernel width parameter $\gamma$

### References

* Cherkassky and Ma, 2004



### Train the DDM

* Use the DDM with the tuned parameters, to re-train the DDM with the entire training dataset




## Test the DDM

### Use the trained DDM to correct the MODFLOW generated residuals

$$ \hat{h}^{new} = \hat{h} + \hat{\epsilon}$$

or 

$$ z_i = M_i + y_i = M_i + \hat{y}(\mathbf{x}_i , \phi) + \varphi_i $$

where $z_i$ is the quantity of interest (head), $M_i$ is the simulated equivalent, $y_i$ is the lumped model residual, $\hat{y}$ is the epistemic term (bias), $\phi$ represents the hyper-parameters and $\varphi_i$ is the aleatoric term (independently distributed random noise), whose distribution is estimated using SVM (parametric approach) .




### Estimate the distribution of the aleatoric term

* Use 10 fold CV once more, and fit a distribution to the CV generalization errors

* Distribution type is chosen on a case-by-case basis - Gaussian is the most widely used noise model; Laplace and Cauchy distributions have pdf that might resemble the shape of the histogram of the aleatoric errors. The Cauchy distribution has heavy tails, which would allow including outliers.

* All 3 distributions have 2 parameters:

 - Gaussian and Laplace: mean and variance
 
 - Cauchy: median and inter-quartile range
 
 * The parameters can be inferred from the CV generalization errors using maximum likelihood estimation (MLE)
 
 * The goodness-of-fit can be assessed by comparing the likelihood corresponding to the estimated parameters (choosing the distribution with the highest likelihood)
 
 * $F(\varphi)$ is the cdf of the fitted distribution.

### Test the DDM

* Use a testing dataset $\{x_i^*, z_i^*\}$

* We have two outputs: $\hat{z}_i^* = M_i^* + \hat{y}_i^*$ and the associated prediction interval $[L_i, U_i]$

* 2 ways to test: does the DDM reduce the the predicitve bias of the PBM, and evaluate the quality of the predicition intervals.

* The DDM-corrected error is $e_i^* = z_i^* - \hat{z}_i^*$. Three statistics to evaluate the error are:

1). PBIAS (percent bias): 

$$ PBIAS = \frac{\sum_{i=1}^m e_i^*}{\sum_{i=1}^m z_i^*} \times 100\%  $$

2). MAE (mean absolute error):

$$ MAE = \frac{1}{m} \sum_{j=1}^m |e_i^*|  $$

3). NSE (Nash-Sutcliffe efficiency) - range of NSE varies from $- \infty$ and $1.0$:

$$ NSE = 1 - \frac{\sum_{j=1}^m (e_i^*)^2}{\sum_{j=1}^m (z_i^* - \bar{z}_i^*)^2}   $$


* The quality of the prediction intervals is evaluated using the prediction interval coverage probability (PICP), defined as the percentage of total observations that fall into the estimated prediction interval:

$$ PICP = \frac{1}{m} \sum_{i=1}^m \mathbf{1} \{ L_i \le z_i^* \le U_i  \}     $$

where $\mathbf{1} \{ L_i \le z_i^* \le U_i  \}  $ is an indicator function that equals $1$ if the observation falls between the interval and $0$ otherwise. A prediction interval with $90\%$ confidence level should theoretically cover $90\%$ os observation data.

## Compute Prediction interval

* Impose the aleatoric error distribution on the DDM-corrected prediction

* The interval $[L_i, U_i]$ can be consructed with a specified confidence level $1 - \alpha$

$$ L_i = M_i + y_i^L = M_i + \hat{y}_i + F^{-1}(\alpha/2)   $$

$$ U_i = M_i + y_i^U = M_i + \hat{y}_i + F^{-1}(1 - \alpha/2)   $$

## Results

* Comparison of pre-DDM and post-DDM based on ME, RMSE; residual plots; hydrographs at representative wells

* Does the DDM remove the global bias - is the ME $\approx$ Zero